## Drop Athena DB and delete S3 bucket

In [13]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

/home/sagemaker-user/.conda/envs/data_science_on_aws/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### List existing S3 buckets

In [14]:
!aws s3 ls

2024-02-18 20:23:31 211125778552personalizepocvod
2024-02-18 01:27:18 aws-athena-query-results-211125778552-us-east-1
2024-03-27 00:35:44 aws-glue-assets-211125778552-us-east-1
2024-03-27 00:28:12 aws-glue-assets-211125778552-us-east-2
2024-02-15 21:19:44 sagemaker-studio-12jvao34qlkn
2024-02-15 22:38:05 sagemaker-studio-211125778552-3pjkfc2ijfr
2024-02-17 02:02:09 sagemaker-studio-211125778552-4dcj21sopi3
2024-02-19 03:02:29 sagemaker-studio-211125778552-4rfwbx1bibn
2024-02-15 20:23:46 sagemaker-studio-211125778552-4yhhjbuzjdq
2024-02-17 02:02:35 sagemaker-studio-211125778552-8xxlet4bnrv
2024-02-17 02:02:08 sagemaker-studio-211125778552-rfcwvtinree
2024-02-20 00:38:45 sagemaker-studio-211125778552-yu1t8p5304s
2024-03-21 19:20:47 sagemaker-studio-uyd2sz3oy3
2024-03-08 03:11:27 sagemaker-team11-stanford-dogs
2024-03-18 01:49:41 sagemaker-team6-distracted-drivers
2024-02-15 20:23:48 sagemaker-us-east-1-211125778552
2024-03-03 20:33:04 team-3-project-data
2024-04-03 23:14:16 team-4-projec

### Drop the Athena database

In [15]:
from pyathena import connect
import pandas as pd

In [16]:
database_name = "team4"
bucket = "team-4-project-data"

In [17]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [18]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [22]:
statement = "DROP DATABASE IF EXISTS {} CASCADE".format(database_name)
print(statement)

DROP DATABASE IF EXISTS team4 CASCADE


In [23]:
pd.read_sql(statement, conn)

""


In [24]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,sagemaker_featurestore
3,team3-court-data


### Remove the Ryanair customer feedback file from the S3 bucket

In [25]:
!aws s3 rm s3://team-4-project-data/ryanair-data/ryanair_reviews.csv

delete: s3://team-4-project-data/ryanair-data/ryanair_reviews.csv


### Remove the S3 bucket

In [26]:
!aws s3 rb s3://team-4-project-data --force

delete: s3://team-4-project-data/athena/staging/30a06c70-3813-4c2c-8dc7-98410acc635c.txt
delete: s3://team-4-project-data/athena/staging/33b42ece-27f7-4411-8ffe-fe9b9fdabeae.txt
delete: s3://team-4-project-data/athena/staging/44a574b6-400c-4377-a659-f96e8cfefb88.txt
delete: s3://team-4-project-data/athena/staging/40d28141-8f3e-41f7-bef2-2fe5aa0068a7.txt
delete: s3://team-4-project-data/athena/staging/dbfa70cd-74ff-45e3-914d-eff5ae312988.txt
delete: s3://team-4-project-data/athena/staging/dc8e9845-cf55-4d55-9b4c-0b79177adb19.txt.metadata
delete: s3://team-4-project-data/athena/staging/dc8e9845-cf55-4d55-9b4c-0b79177adb19.txt
delete: s3://team-4-project-data/athena/staging/40d28141-8f3e-41f7-bef2-2fe5aa0068a7.txt.metadata
delete: s3://team-4-project-data/athena/staging/9ba28089-32c4-43e9-90f9-97c17725070a.txt
delete: s3://team-4-project-data/athena/staging/44a574b6-400c-4377-a659-f96e8cfefb88.txt.metadata
delete: s3://team-4-project-data/athena/staging/9ba28089-32c4-43e9-90f9-97c17725070